## Next step: Naive classification / extraction of carve-out target with LLMs

In [1]:
from dotenv import load_dotenv
from tqdm.auto import tqdm
load_dotenv('../../.env')

True

In [2]:
import pandas as pd
df = pd.read_csv('../../extractions/dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-l0fy7lkzhf/preprocessing_result.csv')
df

,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action,article,company_names,industries
0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,",c18,c181,c33,c333,cacqu,cactio,ccat,ncat,ncdi...",2025-03-14 16:00:00.000,2025-03-14 18:00:07,2025-03-15 07:24:00.694,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add,"Daily Headline News for March 14, 2025\n*** RU...",Central Bank of the Russian Federation,"Energy,Coal Mining,Primary Metals,Non-ferrous ..."
1,M2 Presswire,Impending fintech consolidation could yield un...,The fintech sector is sustaining its rapid exp...,"According to Victor Basta, Managing Partner at...",NaN,621,MTPW,",ifinal,ifmsoft,itech,",NaN,",c18,c181,cacqu,cactio,ccat,ncat,nfact,nfcpin,...",2025-01-28 00:00:00.000,2025-01-28 09:07:17,2025-01-29 07:25:31.631,",eurz,uk,weurz,",MTPW000020250128el1s00001,add,Impending fintech consolidation could yield un...,NaN,"Financial Services,Financial Technology,Techno..."
2,Financial News,UK's private share trading plan will be a flop...,The UK's plan to allow trading of private comp...,The Private Intermittent Securities and Capita...,Private Equity,717,LONFIN,",i81502,i8150203,ialtinv,ifinal,iinv,",bttcap,",c13,c18,c181,cacqu,cactio,ccat,ncat,nfact,nfc...",2025-03-06 00:00:00.000,2025-03-06 00:00:07,2025-03-10 07:18:14.212,",eurz,uk,weurz,",LONFIN0020250306el3600001,add,UK's private share trading plan will be a flop...,Better Capital LLP,"Trusts/Funds/Financial Vehicles,Private Equity..."
3,MFN,VEF initiates a new share buyback program,VEF AB (publ) (“VEF” or the “Company”) has rec...,"Dave Nangle, CEO of VEF comments:\n\n“VEF shar...",NaN,442,MFNEN,",i81502,i8150203,ialtinv,ifinal,iinv,iventure,",zomhfn,",c17,c171,c18,c181,c41,cactio,cboard,ccat,cshb...",2025-04-11 00:00:00.000,2025-04-11 06:00:04,2025-04-12 07:24:33.453,",eecz,eurz,nordz,stock,swed,",MFNEN00020250411el4b0005m,add,VEF initiates a new share buyback program\nDav...,VEF AB,"Trusts/Funds/Financial Vehicles,Private Equity..."
4,CE NoticiasFinancieras,Factop case: Swiss investment fund sues Sauer ...,Nearly two years after the outbreak of the so-...,The vehicle claims to have been defrauded by t...,CE Noticias Financieras English,665,NFINCE,",i81502,ifinal,iinv,",NaN,",c12,c18,c33,c333,cactio,ccat,cliqui,gcat,gcri...",2025-02-18 00:00:00.000,2025-02-18 17:20:30,2025-02-19 07:28:25.421,",eurz,switz,weurz,",NFINCE0020250218el2i008tb,add,Factop case: Swiss investment fund sues Sauer ...,NaN,"Trusts/Funds/Financial Vehicles,Financial Serv..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,Public Companies News and Documents via PUBT,Vinci SA - Determination of the share referenc...,Access the original document here\n\nDetermina...,"Nanterre, 18 February 2025\n\nDetermination of...",NaN,2444,LCDVP,",i831,iconst,icre,ifinal,iinv,","adesmf,banqnp,barbip,crdnat,dwitd,euecar,eurun...",",c17,c172,cactio,ccat,ncat,nfact,nfcpin,npress,",2025-02-18 00:00:00.000,2025-02-18 17:49:17,2025-02-19 07:29:07.855,",eecz,eurz,fra,medz,namz,uk,usa,weurz,",LCDVP00020250218el2i00tbo,add,Vinci SA - Determination of the share referenc...,"Autorité des Marchés Financiers (France),BNP P...","Financial Investment Services,Construction,Rea..."
2907,Public Companies News and Documents via PUBT,Vinpai SA - Details of the Terms and Condition...,Access the original document here\n\nDetails o...,* Camlin Fine Science Limited and certain sign...,NaN,2333,LCDVP,",i25,i41,i831,i83106,ibasicm,icnp,ifinal,ifood...","adesmf,bobexh,camfcm,eparlm,euecar,nseoil,scex...",",c18,c181,cacqu,cactio,ccat,ctendo,ncat,nfact,...",2025-03-13 00:00:00.000,2025-03-13 16:52:33,2025-03-14

## HUGE problem: too many very long articles (30k+ symbols); quick fix - limit len of the articles

In [3]:
df['body'] = df['body'].apply(lambda x: x[:2000])

### Quick fix: dropping duplicated articles

In [4]:
df.drop_duplicates(subset=['body'], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action,article,company_names,industries
0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,",c18,c181,c33,c333,cacqu,cactio,ccat,ncat,ncdi...",2025-03-14 16:00:00.000,2025-03-14 18:00:07,2025-03-15 07:24:00.694,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add,"Daily Headline News for March 14, 2025\n*** RU...",Central Bank of the Russian Federation,"Energy,Coal Mining,Primary Metals,Non-ferrous ..."
1,M2 Presswire,Impending fintech consolidation could yield un...,The fintech sector is sustaining its rapid exp...,"According to Victor Basta, Managing Partner at...",NaN,621,MTPW,",ifinal,ifmsoft,itech,",NaN,",c18,c181,cacqu,cactio,ccat,ncat,nfact,nfcpin,...",2025-01-28 00:00:00.000,2025-01-28 09:07:17,2025-01-29 07:25:31.631,",eurz,uk,weurz,",MTPW000020250128el1s00001,add,Impending fintech consolidation could yield un...,NaN,"Financial Services,Financial Technology,Techno..."
2,Financial News,UK's private share trading plan will be a flop...,The UK's plan to allow trading of private comp...,The Private Intermittent Securities and Capita...,Private Equity,717,LONFIN,",i81502,i8150203,ialtinv,ifinal,iinv,",bttcap,",c13,c18,c181,cacqu,cactio,ccat,ncat,nfact,nfc...",2025-03-06 00:00:00.000,2025-03-06 00:00:07,2025-03-10 07:18:14.212,",eurz,uk,weurz,",LONFIN0020250306el3600001,add,UK's private share trading plan will be a flop...,Better Capital LLP,"Trusts/Funds/Financial Vehicles,Private Equity..."
3,MFN,VEF initiates a new share buyback program,VEF AB (publ) (“VEF” or the “Company”) has rec...,"Dave Nangle, CEO of VEF comments:\n\n“VEF shar...",NaN,442,MFNEN,",i81502,i8150203,ialtinv,ifinal,iinv,iventure,",zomhfn,",c17,c171,c18,c181,c41,cactio,cboard,ccat,cshb...",2025-04-11 00:00:00.000,2025-04-11 06:00:04,2025-04-12 07:24:33.453,",eecz,eurz,nordz,stock,swed,",MFNEN00020250411el4b0005m,add,VEF initiates a new share buyback program\nDav...,VEF AB,"Trusts/Funds/Financial Vehicles,Private Equity..."
4,CE NoticiasFinancieras,Factop case: Swiss investment fund sues Sauer ...,Nearly two years after the outbreak of the so-...,The vehicle claims to have been defrauded by t...,CE Noticias Financieras English,665,NFINCE,",i81502,ifinal,iinv,",NaN,",c12,c18,c33,c333,cactio,ccat,cliqui,gcat,gcri...",2025-02-18 00:00:00.000,2025-02-18 17:20:30,2025-02-19 07:28:25.421,",eurz,switz,weurz,",NFINCE0020250218el2i008tb,add,Factop case: Swiss investment fund sues Sauer ...,NaN,"Trusts/Funds/Financial Vehicles,Financial Serv..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2726,Public Companies News and Documents via PUBT,Vinci SA - Determination of the share referenc...,Access the original document here\n\nDetermina...,"Nanterre, 18 February 2025\n\nDetermination of...",NaN,2444,LCDVP,",i831,iconst,icre,ifinal,iinv,","adesmf,banqnp,barbip,crdnat,dwitd,euecar,eurun...",",c17,c172,cactio,ccat,ncat,nfact,nfcpin,npress,",2025-02-18 00:00:00.000,2025-02-18 17:49:17,2025-02-19 07:29:07.855,",eecz,eurz,fra,medz,namz,uk,usa,weurz,",LCDVP00020250218el2i00tbo,add,Vinci SA - Determination of the share referenc...,"Autorité des Marchés Financiers (France),BNP P...","Financial Investment Services,Construction,Rea..."
2727,Public Companies News and Documents via PUBT,Vinpai SA - Details of the Terms and Condition...,Access the original document here\n\nDetails o...,* Camlin Fine Science Limited and certain sign...,NaN,2333,LCDVP,",i25,i41,i831,i83106,ibasicm,icnp,ifinal,ifood...","adesmf,bobexh,camfcm,eparlm,euecar,nseoil,scex...",",c18,c181,cacqu,cactio,ccat,ctendo,ncat,nfact,...",2025-03-13 00:00:00.000,2025-03-13 16:52:33,2025-03-14

## A Plan

* Classify article as related/unrelated to Carveouts
* Extract PoC target from a list of companies
* do reasoning/verification on a small batch?
* export results to Excel

In [5]:
cols2use = ['source_name', 'title','body','company_names','company_codes','modification_date']
df = df[cols2use]
df

,source_name,title,body,company_names,company_codes,modification_date
0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,Central Bank of the Russian Federation,rosgos,2025-03-15 07:24:00.694
1,M2 Presswire,Impending fintech consolidation could yield un...,"According to Victor Basta, Managing Partner at...",NaN,NaN,2025-01-29 07:25:31.631
2,Financial News,UK's private share trading plan will be a flop...,The Private Intermittent Securities and Capita...,Better Capital LLP,bttcap,2025-03-10 07:18:14.212
3,MFN,VEF initiates a new share buyback program,"Dave Nangle, CEO of VEF comments:\n\n“VEF shar...",VEF AB,zomhfn,2025-04-12 07:24:33.453
4,CE NoticiasFinancieras,Factop case: Swiss investment fund sues Sauer ...,The vehicle claims to have been defrauded by t...,NaN,NaN,2025-02-19 07:28:25.421
...,...,...,...,...,...,...
2726,Public Companies News and Documents via PUBT,Vinci SA - Determination of the share referenc...,"Nanterre, 18 February 2025\n\nDetermination of...","Autorité des Marchés Financiers (France),BNP P...","adesmf,banqnp,barbip,crdnat,dwitd,euecar,eurun...",2025-02-19 07:29:07.855
2727,Public Companies News and Documents via PUBT,Vinpai SA - Details of the Terms and Condition...,* Camlin Fine Science Limited and certain sign...,"Autorité des Marchés Financiers (France),BSE L...","adesmf,bobexh,camfcm,eparlm,euecar,nseoil,scex...",2025-03-14 07:34:31.870
2728,Oslo Stock Exchange (OSE) Filings,Wilh. Wilhelmsen Holding ASA - Wilh. Wilhelmse...,Wilh. Wilhelmsen Holding ASA: Share buy back p...,"DnB Bank ASA,Wilhelm Wilhelmsen Holding ASA","dnc,wilhlm",2025-02-28 07:20:52.620
2729,Securities and Exchange Commission (SEC) Filings,abrdn Global Dynamic Dividend Fund - Annual Re...,UNITED STATES\n\nSECURITIES AND EXCHANGE COMMI...,"AbbVie Inc.,Analog Devices Inc,abrdn Global Dy...","abvbba,adev,agdfun,altddf,amade,amdcs,amtowc,a...",2025-01-11 07:18:16.956


In [6]:
from typing import List, Dict, Any

business_request = """
### Deals search criteria
* Completed date (last 10 years)
* Geography (Europe)
* Deal technique (Divestment)
* Sector (Financial services)
* Size (TBD)
"""

def df_to_prompt_inputs(df: pd.DataFrame) -> List[Dict[str, Any]]:
    """
    Convert a DataFrame with columns ['source_name', 'title', 'body', 'company_names','company_codes','modification_date]
    into a list of dicts compatible with your prompt template keys:
      - news_source
      - article_title
      - article_body
      - mentioned_companies
      - mentioned_company_codes
      - modification_date
    """
    df_renamed = df.rename(columns={
        'source_name': 'news_source',
        'title': 'article_title',
        'body': 'article_body',
        'company_names': 'companies',
        'company_codes': 'company_codes',
        'modification_date': 'modification_date'
    })
    df_renamed['business_request'] = business_request
    return df_renamed[
        ['news_source', 'article_title', 'article_body', 'companies', 'company_codes','modification_date', 'business_request']
    ].reset_index().to_dict(orient='records')
    
def chunk_list(items: List[Any], batch_size: int) -> List[List[Any]]:
    """
    Split a list into sublists (batches) of max length `batch_size`.
    """
    return [items[i:i + batch_size] for i in range(0, len(items), batch_size)]

chunks = chunk_list(df_to_prompt_inputs(df), 20)

In [7]:
chunks[0]

[{'index': 0,
  'news_source': 'Interfax: Russia & CIS Business and Financial Newswire',
  'article_title': 'Daily Headline News for March 14, 2025',
  'article_body': "*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8% IN FEBRUARY - CENTRAL BANK\n\nRussia's broad monetary base declined by 779.6 billion rubles, or 2.8%, to nearly 27.09 trillion rubles in February 2025, the Central Bank of Russia posted on its website.\n\nThe annual growth rate of the monetary base increased to 10.0% as on March 1 from 8.7% on February 1 and 5.5% on January 1.\n\n*** RUBLE'S REAL EFFECTIVE EXCHANGE RATE UP 8.0% IN FEB, 11.0% IN JAN-FEB - CENTRAL BANK\n\nThe real effective exchange rate of the ruble against a basket of currencies of Russia's main trading partners rose 8.0% month-on-month in February 2025, according to a preliminary estimate by the Central Bank of Russia published on its website.\n\nThe real effective exchange rate of the ruble increased 11.0% in January-February 2025.\n\n*** CBR TO LET INDIV

In [8]:
from pydantic import BaseModel,Field
from llm_utils.factory import construct_llm

class CarveOutAssessment(BaseModel):
    is_co: bool = Field(description="Is the article about a future corporate carve-out?")
    co_confidence: float = Field(description="Confidence level of the carve-out assessment (0-1)")
    target_company_code: str = Field(description="Code of the company that may be open to divest/carve-out")
    is_relevant: bool = Field(description="Is the article relevant to the conditions of business request?")
    short_reasoning: str = Field(description="Justification for the answers provided; not more than 2 sentences")


In [9]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_msg = SystemMessagePromptTemplate.from_template(
    "You are an expert in corporate carve-outs. "
    "You are given a news article and a list of companies mentioned in it to understand whether a company is potentially interested in divesting a part of its business. "
    "You are also given a business request that outlines the conditions under which the article is relevant to the business request.\n\n"
    "Your task is to:\n"
    "1. Answer whether the article is about a future corporate carve-out or not.\n"
    "2. If it is about a corporate carve-out, extract the code of the company that is being carved out and the name of the company that may be open to divest.\n"
    "3. Answer whether the article is relevant to the conditions of the business request given to us.\n"
    "Each company mentioned in the article is represented by its code. The codes are unique identifiers for each company.\n\n"
    "Provide justification for your answer."
    "Please note that we are interested only in future carve-outs, not past ones. We are interested in the willingness of the company to divest a part of its business, the fact that it has already done so means it is already late.\n\n"
    "Business request: {business_request}\n"
    "Additional guide: some indicators of a corporate carve-out are: new CEO, management change, strategic shift, legal complications or changes in legislature, IPO, dividends and bonds machuring, focus on core assets, discontinue of operations, M&A activity"
    
)

human_msg = HumanMessagePromptTemplate.from_template(
    "news source: {news_source}\n"
    "article title: {article_title}\n"
    "article body: {article_body}\n"
    "companies: {companies}\n\n"
    "company codes: {company_codes}\n"
    "Outputs:"
)


chat_prompt = ChatPromptTemplate.from_messages([system_msg, human_msg])

In [10]:
llm = construct_llm(model_name='o3-mini')
llm_retry = llm.with_retry(
    retry_if_exception_type=(Exception,),  # which exceptions to retry on
    wait_exponential_jitter=True,          # turn on exp-backoff + jitter
    stop_after_attempt=4,                  # total attempts
)
llm = llm.with_structured_output(CarveOutAssessment)
chain = chat_prompt | llm

In [ ]:
## Test batch
# from langchain.callbacks import get_openai_callback

# def batch_with_usage(batch, chain):
#     # Wrap the call in the OpenAI callback context
#     with get_openai_callback() as cb:
#         results = chain.batch(batch)
#     # After the call, cb has all the stats
#     print(f"Prompt tokens:  {cb.prompt_tokens}")
#     print(f"Completion tokens: {cb.completion_tokens}")
#     print(f"Total tokens:   {cb.total_tokens}")
#     return results

# batch_with_usage(chunks[0], chain)

INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 

In [11]:
import time

from pathlib import Path
from datetime import datetime

def process_and_save(
    chunks: List[List[Dict[str, Any]]],
    chain,
    partial_every: int = 10,
    partial_dir: str = "partials",
    final_path: str = "classification_results.csv",
    max_retries: int = 5,
    initial_delay: float = 5.0,
) -> pd.DataFrame:
    """
    Iterate over each chunk of inputs, run the chain with retry logic, and save results:
      - Shows a progress bar over chunks.
      - Retries on failure with exponential backoff.
      - Saves partial CSV after every `partial_every` chunks into `partial_dir`.
      - Saves the final combined results as `final_path`.
    
    Args:
        chunks: List of batches of prompt-input dicts.
        chain: RunnableSequence with .batch().
        partial_every: Save a partial CSV every N chunks.
        partial_dir: Directory to save partial CSVs.
        final_path: Filepath for the final CSV.
        max_retries: How many times to retry a failed chunk.
        initial_delay: Base delay (in seconds) for exponential backoff.
    
    Returns:
        DataFrame of all successful results.
    """
    partial_dir =f"{partial_dir}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
    Path(partial_dir).mkdir(parents=True, exist_ok=True)
    all_records = []
    failed_chunks = []

    for idx, batch in enumerate(tqdm(chunks, desc="Processing chunks")):
        # Retry logic per chunk
        for attempt in range(1, max_retries + 1):
            try:
                assessments = chain.batch(batch)
                break
            except Exception as e:
                print(f"[Chunk {idx}][Attempt {attempt}] Error: {e}")
                if attempt < max_retries:
                    delay = initial_delay * (2 ** (attempt - 1))
                    print(f"  Retrying in {delay:.1f}s...")
                    time.sleep(delay)
                else:
                    print(f"  Chunk {idx} failed after {max_retries} attempts, skipping.")
                    failed_chunks.append(idx)
        else:
            # All retries failed: skip to next chunk
            continue

        # On success, extend records
        records = [a.dict() for a in assessments]
        all_records.extend(records)

        # Save partials
        if (idx + 1) % partial_every == 0:
            df_partial = pd.DataFrame(all_records)
            df_partial.to_csv(Path(partial_dir) / f"partial_{idx+1}.csv", index=False)
            print(f"Saved partial results to {partial_dir}/partial_{idx+1}.csv")
        time.sleep(5)
    
    # Save final results
    df_final = pd.DataFrame(all_records)
    final_file = Path(f"{final_path}_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv")
    final_file.parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(final_file, index=False)

    if failed_chunks:
        print(f"Warning: the following chunks failed and were skipped: {failed_chunks}")


In [12]:
final_path='../../extractions/dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-l0fy7lkzhf/classification_results_full'

In [ ]:
df_results = process_and_save(
    chunks[:],
    chain,
    partial_every=10,
    partial_dir="partial_outputs",
    final_path="../extractions/classification_results.csv",
    max_retries=4,
    initial_delay=5.0)

Processing chunks:   0%|          | 0/137 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://openai-nc-east-us-2.openai.azure.com/openai/deployments/o3-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 

### FUTURE pipeline improvement: asyncio, logging and other improvement

In [13]:
import asyncio
import logging
from pathlib import Path
from datetime import datetime
import pandas as pd
from typing import List, Dict, Any

logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)


async def process_and_save_async(
    chunks: List[List[Dict[str, Any]]],
    chain,
    *,
    max_concurrent: int = 5,
    partial_every: int = 10,
    partial_dir: str = "partials",
    final_path: str = "classification_results.csv",
    request_timeout: float = 30.0,
) -> pd.DataFrame:
    """
    Async version without manual retry/backoff:
      - Runs up to `max_concurrent` chunk-jobs in parallel.
      - Times out any single chain.batch call after `request_timeout` seconds.
      - Flushes a partial CSV every `partial_every` completed chunks.
      - Saves a timestamped final CSV on completion.
    """
    # Prepare directories
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    partial_path = Path(f"{partial_dir}_{timestamp}")
    partial_path.mkdir(parents=True, exist_ok=True)
    final_path = Path(final_path).with_suffix(f"_{timestamp}.csv")
    final_path.parent.mkdir(parents=True, exist_ok=True)

    all_records: List[Dict[str, Any]] = []
    failed_chunks: List[int] = []
    completed = 0
    lock = asyncio.Lock()

    queue: asyncio.Queue = asyncio.Queue()
    for idx, batch in enumerate(chunks):
        queue.put_nowait((idx, batch))

    async def worker(worker_id: int):
        nonlocal completed
        while True:
            try:
                idx, batch = queue.get_nowait()
            except asyncio.QueueEmpty:
                return

            logger.debug(f"Worker#{worker_id} processing chunk {idx}")
            try:
                # single-shot call with timeout
                assessments = await asyncio.wait_for(
                    chain.batch(batch),
                    timeout=request_timeout
                )
                records = [a.dict() for a in assessments]

                async with lock:
                    all_records.extend(records)
                    completed += 1

                    if completed % partial_every == 0:
                        df_partial = pd.DataFrame(all_records)
                        path = partial_path / f"partial_{completed}.csv"
                        df_partial.to_csv(path, index=False)
                        logger.info(f"Saved partial results to {path}")

            except Exception as e:
                logger.error(f"Chunk {idx} failed: {e}")
                async with lock:
                    failed_chunks.append(idx)

            finally:
                queue.task_done()

    # Launch workers
    tasks = [asyncio.create_task(worker(i)) for i in range(max_concurrent)]

    # Run until done (or cancelled)
    try:
        await queue.join()
    except asyncio.CancelledError:
        logger.warning("Cancellation requested—shutting down workers")
    finally:
        for t in tasks:
            t.cancel()
        await asyncio.gather(*tasks, return_exceptions=True)

    # Final save
    df_final = pd.DataFrame(all_records)
    df_final.to_csv(final_path, index=False)
    logger.info(f"Saved final results to {final_path}")

    if failed_chunks:
        logger.warning(f"Chunks skipped due to failure: {failed_chunks}")

    return df_final

# In Jupyter:
# import nest_asyncio; nest_asyncio.apply()
# results = await process_and_save_async(chunks, chain, max_concurrent=8)

In [ ]:
import nest_asyncio; nest_asyncio.apply()
results = await process_and_save_async(chunks, chain[100:], max_concurrent=5, partial_every=5,final_path=final_path)